In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, HTML
from IPython.core.interactiveshell import InteractiveShell
from ipyevents import Event
import random
import threading

In [2]:
# Setup global configurations
InteractiveShell.ast_node_interactivity = "all"
height = 10
width = 20
pd.set_option('display.max_rows', height)
pd.set_option('display.max_columns', width)

In [3]:
# Load empty board
df = pd.DataFrame(np.array(list(" " * height * width)).reshape(height,width))
disk = {"x":height//2-1, "y":width//3, "img":"💾"}
df.iloc[(disk["x"], disk["y"])] = disk["img"]

In [4]:
# Initialize board
output = widgets.Output()
board = widgets.HBox([output], layout=Layout(height='350px'))
def refreshBoard():
    output.clear_output(wait=True)
    with output:
        display(df.style.set_properties(**{'height': '30px', 'width':'30px', 'font-size':'30px', 
                                           'padding':'0px', 'margin':'0px', 'line-height':'1'})
)

In [5]:
# Call this function to constantly refresh the board
boardSpeed = 0.2
def startRefresh():
    refreshBoard()
    timer = threading.Timer(boardSpeed, startRefresh)
    timer.start()

In [6]:
diskDropSpeed = 0.5
def dropDisk():
    global disk
    nextX = disk["x"] + 1
    if nextX < height and df.iloc[(nextX, disk["y"])] == " ":
        df.iloc[(disk["x"], disk["y"])] = " "
        disk["x"] = nextX
        df.iloc[(disk["x"], disk["y"])] = disk["img"]
    dropDiskTimer = threading.Timer(diskDropSpeed, dropDisk)
    dropDiskTimer.start()

In [7]:
def jumpDisk():
    global disk
    nextX = disk["x"] - 1
    if nextX >= 0 and df.iloc[(nextX, disk["y"])] == " ":
        df.iloc[(disk["x"], disk["y"])] = " "
        disk["x"] = nextX
        df.iloc[(disk["x"], disk["y"])] = disk["img"]
        refreshBoard()

In [8]:
# Listen to keyboard events
keyPressed = HTML('Key Pressed')
boardEvent = Event(source=output, watched_events=['keydown'])
def handleEvent(event):
    lines = ['{}: {}'.format(k, v) for k, v in event.items()]
    content = '<br>'.join(lines)
    keyPressed.value = event['code']
    if event['code'] == 'Space':
        jumpDisk()
boardEvent.on_dom_event(handleEvent)

In [9]:
stackSpeed = 5.0
stackCountDown = 10
def randomCdStack():
    global stackSpeed, stackCountDown
    length = random.randrange(3,6)
    if random.randrange(2) == 0:
        # ceiling stack
        for i in range(length):
            df.iloc[(i, width-1)] = "💽"
    else:
        # floor stack
        for i in range(length):
            df.iloc[(height-1-i, width-1)] = "💽"
    stackCountDown -= 1
    if stackCountDown == -1:
        stackCountDown = 10
        stackSpeed *= 0.8
    stackTimer = threading.Timer(stackSpeed, randomCdStack)
    stackTimer.start()

In [10]:
diskMoveSpeed = 1.0
def moveDiskRight():
    global df,disk
    nextY = disk["y"] + 1
    if df.iloc[(disk["x"], nextY)] == " ":
        df.iloc[(disk["x"], disk["y"])] = " "
        df.drop(0, inplace=True, axis=1)
        line = pd.DataFrame(np.array(list(" " * height)))
        df = pd.concat([df,line], axis=1)
        df.columns = range(width)
        df.iloc[(disk["x"], disk["y"])] = disk["img"]
    moveDiskTimer = threading.Timer(diskMoveSpeed, moveDiskRight)
    moveDiskTimer.start()

In [11]:
# Start game 
def startGame():
    display(board)
    startRefresh()
    dropDisk()
    randomCdStack()
    moveDiskRight()

In [12]:
startGame()